# World Bank Data - CSV Imports

Parsing data from CSV bulk files. Used before I discovered API and managed to automate it in Lambda Function.

Saved at: https://databank.worldbank.org/data/embed/selected-indicators/id/b8dbe5a6

In [37]:
import pandas as pd
import numpy as np

from SqlAlquemyInsertIndicatorsHandler import SqlAlquemyInsertIndicatorsHandler

from dotenv import load_dotenv
load_dotenv()

True

In [38]:
all_data = pd.read_csv('RawData/9636c3fb-b687-4862-b6ed-8b712fea256b_Data.csv', index_col=2)

In [39]:
unique_indicators = all_data['Series Name'].unique()
unique_indicators

array(['Current account balance (% of GDP)',
       'Current account balance (BoP, current US$)', 'Expense (% of GDP)',
       'GDP (current US$)', 'GDP per capita (current US$)',
       'Listed domestic companies, total',
       'Market capitalization of listed domestic companies (% of GDP)',
       'Market capitalization of listed domestic companies (current US$)',
       'Population, total', 'Stocks traded, total value (% of GDP)',
       'Stocks traded, total value (current US$)'], dtype=object)

In [40]:
all_countries = [
    'United States', 'Japan', 'United Kingdom', 'Canada', 'France',
    'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
    'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland',
    'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria',
    'Portugal', 'Eurozone', 'China', 'Taiwan', 'India', 'Korea',
    'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand',
    'Indonesia', 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait',
    'Turkiye', 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru',
    'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia']

selected_countries = [
    'United States', 'Japan', 'United Kingdom', 'Canada', 'France',
    'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Sweden',
    'Spain', 'Italy', 'Belgium', 'Norway', 'Israel', 'Ireland',
    'Austria', 'China', 'Taiwan', 'India', 'Korea', 'Brazil',
    'Saudi Arabia', 'South Africa', 'Mexico', 'Indonesia',
    'Turkiye',  'Poland', 'Argentina', 'Russia']

In [41]:
def check_data_coverage(df):
    total_na = df.isna().sum().sum()
    total_cells = len(df.index)*len(all_countries)
    data_coverage_pct = round((total_cells-total_na)/total_cells*100, 1)
    print('Data coverage since 1999 (50 countries):', data_coverage_pct, '%')

    df_recent = df['2013':]    
    df_recent = df_recent[selected_countries]
    nas = df_recent.isna().sum().sum()
    total = df_recent.shape[0]*len(selected_countries)
    coverage_11y = round((total - nas)/total*100, 1)
    print('Data coverage since 2013 (30 countries):', coverage_11y, '%')
    print()
    
    available_countries = df.columns[~df.isna().all()]
    print('Available countries:')
    print(available_countries)
    print()

    no_data = df.columns[df.isna().all()]
    print('No data:')
    print(no_data)
    print()

    print('Missing first:')
    for country in df.columns:
        first_valid = df[country].first_valid_index()
        if first_valid != None and (not str(first_valid).startswith('1999')):
            print(country, first_valid)
    print()

    print('Missing last:')
    for country in df.columns:
        last_valid = df[country].last_valid_index()
        if last_valid != None and (not str(last_valid).startswith('202')):
            print(country, last_valid)
            
    print()
    print('Done')

In [42]:
def save_to_database(indicator, df):
    sql_handler = SqlAlquemyInsertIndicatorsHandler()
    indicator_id = sql_handler.get_indicator_id(indicator, 'World Bank')

    sql_handler.delete_all_records(indicator_id)
    initial_count = sql_handler.get_indicator_count(indicator_id)
    for country in df.columns:
        values = df[country].dropna()
        if (len(values) > 0):
            sql_handler.insert_into_table(indicator_id, country, values)

    final_count = sql_handler.get_indicator_count(indicator_id)
    print(f'{indicator}: Inserted', (final_count-initial_count), 'records')

In [43]:
def read_world_bank_indicator(all_data, indicator, filename):
    all_data.index.name = None
    df_indicator = all_data[all_data['Series Name']
                             == indicator].iloc[:, 3:].T
    df_indicator.index = pd.date_range(
        start='01-01-1999', end='01-01-2022', freq='YS')

    df_indicator.rename(columns={
        "Egypt, Arab Rep.": 'Egypt',
        "Hong Kong SAR, China": 'Hong Kong',
        "Korea, Rep.": 'Korea',
        'Russian Federation': 'Russia',
        'Euro area': 'Eurozone'
    }, inplace=True)
    if 'Taiwan' not in df_indicator.columns:
        df_indicator.loc[:, 'Taiwan'] = np.nan
        
    df_indicator = df_indicator[all_countries]

    df_indicator[df_indicator == '..'] = None
    check_data_coverage(df_indicator)
    df_indicator.to_csv(f'Indicators/WorldBank_{filename}.csv')
    return df_indicator

## 1. Stock Market Indicators

In [46]:
df = read_world_bank_indicator(all_data, indicator='Market capitalization of listed domestic companies (current US$)',
                               filename='Stock Market Cap')
save_to_database('Stock Market Cap', df)
df

Data coverage since 1999 (50 countries): 76.2 %
Data coverage since 2013 (30 countries): 67.0 %

Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland',
       'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria',
       'Portugal', 'Eurozone', 'China', 'India', 'Korea', 'Brazil',
       'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia',
       'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye',
       'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary',
       'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Taiwan'], dtype='object')

Missing first:
China 2003-01-01 00:00:00
India 2000-01-01 00:00:00
Brazil 2000-01-01 00:00:00
Saudi Arabia 2009-01-01 00:00:00
United Arab Emirates 2007-01-01 00:00:00
Qatar 200

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,14777387400000,4455348090000,2954815470000,1448278090000,1502951790000,678232020000,1432166980000,427835360000,694055330000,97452590000,...,29576810000,68227520000,None,12091970000,None,10582930000,None,None,55847580000,None
2000-01-01,15107751000000,3157221780000,2576991340000,770841540000,1446634120000,792315950000,1270243170000,372794350000,640456300000,111818640000,...,31279370000,60400770000,None,9749910000,None,9746160000,None,None,45839340000,None
2001-01-01,13983666000000,2264527900000,2149500800000,615265930000,1174663000000,527355900000,1071748730000,375130790000,503022880000,85145500000,...,26016650000,56309720000,84752030000,9790330000,None,8150100000,None,None,33384000000,None
2002-01-01,11054430000000,2069299120000,1856194400000,888677700000,967015060000,612667140000,686013540000,380087000000,401267710000,76749870000,...,28380430000,49827000000,67061460000,11441440000,12988880000,10256370000,None,None,16571000000,None
2003-01-01,14266265650000,2953098300000,2425822000000,910230610000,1355925340000,727102780000,1079026240000,585529690000,488647360000,121640960000,...,37020270000,86525910000,106643670000,14125030000,16662910000,15507730000,None,None,34994620000,None
2004-01-01,16323726330000,3557674430000,2815928030000,1177517940000,1559109680000,829098070000,1194516790000,776402760000,538663690000,151349870000,...,70530890000,116924270000,125241630000,18073730000,28597850000,26891360000,None,None,40593870000,None
2005-01-01,17000864470000,4572901030000,3058182420000,1482184560000,1758513210000,938697660000,1202136280000,804014820000,592835610000,None,...,94028510000,136493350000,145120740000,24139720000,32575670000,34886410000,None,50500810000,47590300000,None
2006-01-01,19568972500000,4614068830000,3781358500000,1700708090000,2428251770000,1212534300000,1637609770000,1095857960000,779542640000,None,...,148848930000,174418780000,200696040000,40021560000,41934490000,44372130000,93496400000,56204320000,51240130000,None
2007-01-01,19922279820000,4330921860000,3846461880000,2186550150000,2740341210000,1274656700000,2105197820000,1298315040000,956157630000,None,...,211620200000,212910230000,264960820000,69386470000,46195630000,68913190000,139273790000,101955950000,57070160000,None
2008-01-01,11590277780000,3115803730000,1868152970000,1033651810000,1472406870000,880334430000,1110579630000,683871560000,388720840000,None,...,90815490000,131807960000,90200000000,37876760000,18465410000,40912350000,85978070000,87716200000,39850360000,None


In [47]:
df = read_world_bank_indicator(all_data, indicator='Market capitalization of listed domestic companies (% of GDP)',
                               filename='Stock Market Cap Pct of GDP')
save_to_database('Stock Market Cap Pct of GDP', df)
df

Data coverage since 1999 (50 countries): 76.2 %
Data coverage since 2013 (30 countries): 67.0 %

Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland',
       'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria',
       'Portugal', 'Eurozone', 'China', 'India', 'Korea', 'Brazil',
       'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia',
       'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye',
       'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary',
       'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Taiwan'], dtype='object')

Missing first:
China 2003-01-01 00:00:00
India 2000-01-01 00:00:00
Brazil 2000-01-01 00:00:00
Saudi Arabia 2009-01-01 00:00:00
United Arab Emirates 2007-01-01 00:00:00
Qatar 200

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,153.432869655488,96.1036533411112,174.902449547038,213.481242781549,100.656333315236,227.684361486859,65.2484138800771,109.873326727906,155.252448378671,54.7596693896892,...,17.3949875356791,90.2521030550515,None,24.0936732613521,None,16.238202991434,None,None,19.6977246995835,None
2000-01-01,147.379061960136,63.5465690563746,154.676825247855,103.500141588137,105.93088070984,283.77162954461,65.2081577684527,89.6474345629142,153.410299039009,68.1165386625965,...,18.1624015472295,77.1895968604588,None,18.8423176521729,None,15.7644922547102,None,None,16.1290412248255,None
2001-01-01,132.146652819017,51.7640691192871,130.370339041825,83.2607526492622,85.2652518516862,184.01691311301,55.0803629181818,98.8857398123606,116.551947131402,51.6687503414718,...,13.6280258455015,78.7358378718759,62.1762660131815,18.8166444816383,None,12.0192549510764,None,None,12.4244152562322,None
2002-01-01,101.146635097615,49.4710877143585,103.94298140457,116.831457041084,64.4071544355359,198.08188509019,33.0054678893422,96.0851664954883,84.6802923374213,42.9646695123394,...,14.2564767923642,70.8824741290226,43.3874457628042,20.887095654701,19.2117847236461,12.4779116797678,None,None,16.9569392155608,None
2003-01-01,124.526145193632,65.3403497735733,117.947031170342,101.640345797861,73.510025121304,200.808136571689,43.132747814909,125.247538629914,84.2393256239939,55.7738101157349,...,16.9951326975856,113.094153889894,52.6973346341978,24.0503699163637,19.5378967584842,15.4937132124081,None,None,27.4280508755432,None
2004-01-01,133.612737929627,72.7077474943828,116.214321336038,114.690672619493,73.5556365841485,205.265865561371,42.4437311528287,126.382784095783,81.8165228716809,60.2088424432982,...,27.6475415635864,118.010717225024,51.975339689439,27.0691642240112,27.4660290139012,22.4441302309328,None,None,24.6534557360052,None
2005-01-01,130.382734540376,94.6483020695092,120.173160872401,126.346747568222,80.0435615798352,224.411674114104,42.226681387933,115.634839352651,86.501376379037,None,...,30.7136257319987,111.591704774505,58.5458367390391,31.7374804780876,28.7742573180902,25.4379110663214,None,34.6844965843913,23.946359886693,None
2006-01-01,141.644163028722,100.26960187836,139.534648208639,128.913321846832,104.641838714103,274.565445936082,54.6835335229854,146.523207674162,106.211191876511,None,...,43.1913604091568,113.376703883859,73.3681009896472,45.1490504997054,36.2392652803656,28.395580624,87.033236198802,34.7384214129495,22.0333391526542,None
2007-01-01,137.639681557832,94.5667790247779,124.360370899668,148.864363431601,102.99745268657,259.733985642434,61.4552517757464,151.951500384826,112.680173885233,None,...,49.3263314933457,123.379113179655,83.0851267056093,67.9121108782488,32.9529355415374,36.2349876732928,106.774079068206,49.43809291479,19.8483841980828,None
2008-01-01,78.4725069790145,61.014281060687,63.722868756244,66.5588326938396,50.2475845553422,155.187083153931,29.6529057027698,64.7798197864769,40.8375976627339,None,...,17.0193993553643,73.3638385417982,25.3435790321484,31.4198021892782,11.6629350374689,17.2759383641779,52.8061848129537,36.1710097864041,11.0218432201051,None


In [48]:
df = read_world_bank_indicator(all_data, indicator='Listed domestic companies, total',
                               filename='Listed domestic companies total')
save_to_database('Listed Domestic Companies Total', df)
df

Data coverage since 1999 (50 countries): 80.5 %
Data coverage since 2013 (30 countries): 65.7 %

Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland',
       'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria',
       'Portugal', 'Eurozone', 'China', 'India', 'Korea', 'Brazil',
       'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia',
       'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye',
       'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary',
       'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Taiwan'], dtype='object')

Missing first:
Saudi Arabia 2002-01-01 00:00:00
United Arab Emirates 2003-01-01 00:00:00
Qatar 2008-01-01 00:00:00
Russia 2000-01-01 00:00:00

Missing last:
United States 2019-0

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,7229,1889,2292,1538,1144,239,617,1217,387,233,...,221,282,262,229,64,74,1032,85,125,None
2000-01-01,6917,2055,2428,1507,1185,252,744,1333,392,225,...,225,260,309,209,58,57,1075,73,122,21
2001-01-01,6177,2103,2438,1278,936,263,749,1334,329,208,...,230,249,313,204,55,47,1109,120,116,21
2002-01-01,5685,2119,2405,1252,874,258,715,1355,294,193,...,202,245,313,198,48,44,1150,110,110,57
2003-01-01,5295,2174,2311,3578,817,289,684,1405,268,188,...,188,239,339,195,50,37,977,110,106,266
2004-01-01,5226,2276,2486,3597,787,282,660,1514,256,178,...,211,239,319,192,47,33,795,106,103,412
2005-01-01,5145,2323,2757,3719,749,284,648,1643,237,None,...,234,245,302,193,44,23,744,98,100,414
2006-01-01,5133,2391,2913,3790,730,256,656,1751,224,None,...,253,244,288,189,41,15,595,94,101,539
2007-01-01,5109,2389,2588,3881,707,257,761,1913,221,None,...,352,238,289,188,39,13,435,90,106,592
2008-01-01,4666,2374,2415,3755,673,253,742,1924,211,None,...,432,235,289,201,40,19,373,89,107,561


In [49]:
df = read_world_bank_indicator(all_data, indicator='Stocks traded, total value (current US$)',
                               filename='Stocks traded total value')
save_to_database('Stocks Traded Total Value', df)
df

Data coverage since 1999 (50 countries): 78.2 %
Data coverage since 2013 (30 countries): 63.7 %

Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland',
       'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria',
       'Portugal', 'Eurozone', 'China', 'India', 'Korea', 'Brazil',
       'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia',
       'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye',
       'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary',
       'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Taiwan'], dtype='object')

Missing first:
China 2000-01-01 00:00:00
India 2000-01-01 00:00:00
Saudi Arabia 2001-01-01 00:00:00
United Arab Emirates 2006-01-01 00:00:00
Qatar 2006-01-01 00:00:00
Kuwait 200

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,18815724700000,2028693270000,1423785660000,392672780000,723606290000,465251450000,1308493540000,198808400000,419908790000,55562340000,...,10745410000,6521940000,183894700000,2455010000,None,3933790000,None,1382000000,11862190000,None
2000-01-01,29782045700000,2481024210000,1833383630000,628487350000,1080301380000,634098120000,1827236880000,213223310000,650382120000,89909810000,...,20459530000,5763660000,91619800000,2190650000,None,6650030000,None,1179000000,9145630000,None
2001-01-01,20883579600000,1701945370000,1893938290000,445950640000,980888380000,586594590000,1301049770000,240570260000,455908330000,66026060000,...,10176860000,4071120000,37130260000,736170000,None,3448100000,None,784000000,6662000000,28201700000
2002-01-01,17052861300000,1732589850000,1047574930000,448461700000,1032545060000,490762350000,871757290000,290946000000,528096140000,32507890000,...,3242530000,2661800000,20448960000,959820000,6642220000,5922260000,None,1031000000,1168610000,47667400000
2003-01-01,16045083400000,2327856190000,1245706210000,506548290000,1103021950000,448671720000,1017280520000,421669680000,538915870000,43877390000,...,7764010000,7562670000,33877540000,969200000,8862590000,8768390000,None,805800000,2929510000,79722900000
2004-01-01,19103123800000,3457685670000,1621275060000,706494160000,1323735230000,43853810000,1181299170000,544922320000,628501280000,68190070000,...,17805410000,13081570000,39892350000,1057210000,14269590000,18009920000,None,2079600000,4551450000,118693700000
2005-01-01,25792191700000,4337679650000,1729902330000,932109740000,1271455120000,56648390000,1288068410000,612669600000,594702360000,None,...,26241350000,20450590000,49161710000,1715940000,22164080000,34340240000,None,9548720000,5257770000,147885600000
2006-01-01,30581777000000,5498545140000,2763295580000,1237142820000,2015577290000,81267490000,1950838200000,858943720000,900783940000,None,...,53233690000,27812170000,86545360000,4132310000,30986590000,28778550000,48363770000,15617180000,3387990000,582649200000
2007-01-01,42852575300000,6585830110000,3947700220000,1744996560000,2895762310000,1259262900000,3254836700000,1373259640000,1541464470000,None,...,90168840000,45162640000,121523700000,8060850000,47286400000,37788550000,61517870000,17247590000,4845000000,1277021300000
2008-01-01,47245495800000,6208708320000,2735562940000,1502829750000,2405976410000,1412767510000,4189639200000,839733940000,970763610000,None,...,51688940000,28872350000,81801420000,3681120000,27563770000,32280700000,95826680000,18034040000,3337600000,1155013100000


In [50]:
df = read_world_bank_indicator(all_data, indicator='Stocks traded, total value (% of GDP)',
                               filename='Stocks traded total value Pct of GDP')
save_to_database('Stocks Traded Total Value Pct of GDP', df)
df

Data coverage since 1999 (50 countries): 78.2 %
Data coverage since 2013 (30 countries): 63.7 %

Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland',
       'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria',
       'Portugal', 'Eurozone', 'China', 'India', 'Korea', 'Brazil',
       'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia',
       'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye',
       'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary',
       'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Taiwan'], dtype='object')

Missing first:
China 2000-01-01 00:00:00
India 2000-01-01 00:00:00
Saudi Arabia 2001-01-01 00:00:00
United Arab Emirates 2006-01-01 00:00:00
Qatar 2006-01-01 00:00:00
Kuwait 200

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,195.362722599304,43.7597311853416,84.2772085405203,57.8813376102966,48.4616714919654,156.186196169396,59.6139481286792,51.0564163968406,93.9289202536999,31.2210826712508,...,6.31969008881492,8.62729293104692,128.968476412453,4.89169331327748,None,6.03591638097136,None,1.60178124341213,4.1838545726449,None
2000-01-01,290.529673047955,49.936490774906,110.043815416852,84.3863833692113,79.1058879597386,227.105180456829,93.8015283730085,51.2747114609247,155.787858623336,54.7703410541543,...,11.879849220991,7.36571060005944,70.229476857361,4.23356966010278,None,10.7564770564602,None,1.18047470879629,3.21798357692325,None
2001-01-01,197.351334265243,38.9041873893144,114.870102393818,60.3481911160271,71.1997353786511,204.687812728732,66.8648270814531,63.4151308586318,105.635361109073,40.0665215445448,...,5.33083664138351,5.69249934606231,27.2397124045122,1.41489093503975,None,5.08504104204938,None,0.798365459233446,2.47937498313619,9.19814401218018
2002-01-01,156.031522139209,41.4213216513412,58.6619922293075,58.9577456912909,68.7717202057532,158.668753508294,41.9419669798281,73.5505156745596,111.445138502332,18.1979559104593,...,1.62883556357479,3.78660103230442,13.2300749626649,1.75221407019529,9.82447298974943,7.20502840913713,None,1.05262284382902,1.19582697101542,13.7978208762292
2003-01-01,140.052935654702,51.5062223554081,60.5680668985433,56.5634057933025,59.7991414880186,123.912237036991,40.6645385437332,90.197116280924,92.9052588289191,20.1182991176167,...,3.56427384822913,9.88482831094742,16.7403847032217,1.65023497457631,10.3917244006464,8.76046461955082,None,0.851392321503974,2.29608863649362,18.5252417602361
2004-01-01,156.362623479894,70.6642897082929,66.9105810907195,68.8127863360994,62.4511468054731,10.857208083736,41.9740809022364,88.7024151196128,95.461770125996,27.126849734509,...,6.97957750188174,13.2031224837184,16.555345393221,1.58339153618356,13.7048405022222,15.0314818562052,None,1.7760330339678,2.76418511242315,20.08296920529
2005-01-01,197.805028654262,89.7797724246753,67.9775770199579,79.4563897128041,57.8737740580958,13.5427631041167,45.2451650115172,88.1152299807768,86.7737561781445,None,...,8.57151732599382,16.7196145581044,19.8332329856641,2.25601673306773,19.577646173931,25.03966361446,None,6.55816305174726,2.64559064812489,19.3563490016175
2006-01-01,221.357059349745,119.49040042777,101.967448114688,93.7751701558359,86.8582559278802,18.4021554128045,65.1429467886116,114.846260792756,122.730086824525,None,...,15.4467720439732,18.0786275563763,31.6382361737948,4.66173414705568,26.7782261127755,18.4165970118364,45.0205079326534,9.65257083658138,1.45684120465348,58.8574937616118
2007-01-01,296.061237544901,143.80327349224,127.633518510237,118.802581360749,108.83905275277,256.597224953707,95.0156830807661,160.722826345697,181.656746824798,None,...,21.0173607822431,26.1712481924989,38.106811460783,7.88956894583241,33.7310193884434,19.8694566808126,47.162599046724,8.36329764938881,1.68503858127805,98.2548203877637
2008-01-01,319.877794929994,121.580146661005,93.3103022072464,96.7701047198619,82.1067230551794,249.045432713045,111.864987228541,79.5439033928947,101.98489417445,None,...,9.6868354959651,16.0702466203277,22.9838220921504,3.05358911995101,17.4095489294705,13.6310767666124,58.8550240089336,7.43659024591127,0.923115975148604,69.5435740931097


## 2. GDP Indicators

In [51]:
df = read_world_bank_indicator(all_data, indicator='GDP (current US$)',
                               filename='GDP Current US Dollars')
save_to_database('GDP', df)
df

Data coverage since 1999 (50 countries): 98.0 %
Data coverage since 2013 (30 countries): 96.7 %

Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland',
       'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria',
       'Portugal', 'Eurozone', 'China', 'India', 'Korea', 'Brazil',
       'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia',
       'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye',
       'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary',
       'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Taiwan'], dtype='object')

Missing first:

Missing last:

Done
GDP: Inserted 1176 records


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,9631174489000,4635982020564.97,1689407711357.04,678409995711.893,1493151737698.46,297882566712.49,2194945278872.59,389389647825.542,447049523049.808,177964167947.204,...,170030647848.034,75596598517.3586,142588875293.749,50187324567.883,49073373275.9379,65173036730.6204,90710704806.8416,86278947620.5906,283523000000,195907128350.934
2000-01-01,10250947997000,4968359152795.66,1666048767079.76,744773415931.587,1365639660792.16,279209007352.669,1947981991011.77,415844972940.496,417479337444.707,164157842126.821,...,172220451786.957,78249883995.6255,130457756628.436,51744749133.213,47218411470.2538,61823494487.0363,99838543960.0763,99875074935.083,284203750000,259710142196.943
2001-01-01,10581929774000,4374709984220.01,1648765214387,738962729044.525,1377657339291.34,286580125206.282,1945790973803.15,379357823192.53,431586852369.686,164791096044.098,...,190905493539.168,71517267767.736,136309295225.34,52030158775.4055,53749989092.0197,67808695573.681,96684636118.5984,98200641189.1568,268696750000,306602070620.5
2002-01-01,10929112955000,4182845406488.62,1785781372553.92,760649334098.005,1501409382971.38,309299934075.77,2078484517474.51,395573025330.447,473861980070.981,178634843165.633,...,199070432431.108,70295232513.0515,154564203586.954,54777553515.0809,67608919144.3684,82196206089.7585,85146067415.7303,97945812789.8533,97724004251.8602,345470494417.863
2003-01-01,11456442041000,4519563042183.73,2056704586736.55,895540646634.787,1844544792036.86,362088305988.747,2501640388482.35,467497961560.862,580070360701.96,218096916361.973,...,217828661056.935,76507853875.6649,202370140236.265,58731030121.8671,85285075491.886,100090467581.268,80288461538.4615,94644969146.1352,127586973492.177,430347420194.66
2004-01-01,12217193198000,4893116005656.56,2423047347028.45,1026690238278.25,2119633181634.37,403914244451.966,2814353869359.08,614326362213.686,658380081545.175,251374821136.171,...,255107275407.423,99079365628.3333,240963562236.127,66768703497.5687,104120803140.221,119814667457.855,78782467532.4675,117092416651.395,164657930452.787,591016690742.798
2005-01-01,13039199193000,4831466523975.98,2544813166100.5,1173108598778.68,2196945232435.8,418292704114.275,2846864211175.1,695305000206.729,685348181515.953,264466895451.157,...,306145913284.466,122314960843.922,247875422204.414,76060606060.6061,113211158292.936,137143375920.47,89600665557.4043,145600527535.771,198737095012.282,764015982495.676
2006-01-01,13815586948000,4601662661030.07,2709978165671.04,1319264809590.97,2320536221304.7,441619409123.416,2994703642023.53,747907432136.598,733955269898.823,282885978374.285,...,344626630395.377,153840051814.059,273546728473.073,88643193061.748,115715618613.052,156264210926.177,107426086956.522,161792959247.85,232557260817.308,989932059220.667
2007-01-01,14474226905000,4579749785984.82,3092996468387.23,1468820407783.26,2660591246211.77,490754683815.144,3425578382921.58,854427259166.209,848558887541.179,319423375611.047,...,429020755432.721,172565861848.899,318902829550.733,102170981144.136,140186691233.532,190184113270.15,130437828371.278,206229536757.674,287530508430.568,1299703459799.96
2008-01-01,14769857911000,5106679413137.57,2931683721186.75,1552989690721.65,2930303780828.12,567273005013.434,3745264093617.19,1055686110665.53,951869997864.062,353358901702.046,...,533599853342.51,179663390874.653,355908689477.445,120550599815.441,158325583917.574,236816948159.718,162818181818.182,242504150472.931,361558037110.419,1660848058303.11


In [52]:
df = read_world_bank_indicator(all_data, indicator='GDP per capita (current US$)',
                               filename='GDP per capita Current US Dollars')
save_to_database('GDP Per Capita', df)
df

Data coverage since 1999 (50 countries): 98.0 %
Data coverage since 2013 (30 countries): 96.7 %

Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland',
       'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria',
       'Portugal', 'Eurozone', 'China', 'India', 'Korea', 'Brazil',
       'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia',
       'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye',
       'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary',
       'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Taiwan'], dtype='object')

Missing first:

Missing last:

Done
GDP Per Capita: Inserted 1176 records


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,34515.3902272076,36610.1667092968,28788.9692869593,22315.1742893999,24678.3910613218,41696.9403674346,26734.9425369252,20698.7127028167,28272.6432492539,33440.6030643404,...,4398.07180472258,4981.19110628657,13249.6633239242,1911.73501688306,4793.47784826397,6337.40995410482,1297.57469006096,2236.07292549395,7735.32208018486,1330.75720214844
2000-01-01,36329.9560727102,39169.3601759313,28289.65268964,24271.0020563821,22416.425417915,38864.0438950021,23694.7604830673,21853.4499933573,26214.498549887,30743.3796974953,...,4501.47996121234,5097.11493718915,12072.9293569196,1941.318259717,4624.28220296128,6028.5826120265,1398.85983078672,2546.85021319149,7666.51783423783,1771.59411621094
2001-01-01,37133.623113437,34406.1690160364,27888.6051075925,23821.4455867378,22449.3396931175,39638.4387853866,23628.3272122367,19681.6450326534,26896.5481111196,30751.5896881994,...,4991.24435799511,4606.89056424429,12549.036894906,1925.97942030475,5276.03318905496,6637.1065117699,1327.09651832991,2465.00701127148,7168.97587232911,2100.3525390625
2002-01-01,37997.7596573051,32820.788626377,30078.6081337481,24255.3385818322,24288.2700193508,42458.5341569947,25197.2656001848,20290.7804189053,29343.2449960602,33228.6339176661,...,5207.12887879142,4479.17504395379,14177.5721592705,2003.97108061855,6655.33300865319,8060.88881086776,1144.53239841974,2421.16210341989,2579.48876933284,2377.52954101563
2003-01-01,39490.2749557007,35387.0483579741,34480.941057112,28300.4630963791,29627.92426353,49337.5468934732,30310.3575963774,23705.9072163917,35750.9746630269,40458.9411743486,...,5701.63886301914,4824.22054120463,18518.3788387396,2126.13782387391,8419.43212215959,9818.56849307488,1056.93720896595,2305.17050671011,3333.15290388997,2975.123046875
2004-01-01,41724.6316287624,38298.9801712303,40392.2648578651,32143.6814078562,33797.1624418436,54659.6403000107,34106.6581224001,30819.9934867745,40436.6182310407,46511.9347509801,...,6681.31035976437,6185.51479184893,21995.477943746,2393.66589710452,10301.7017009768,11749.8755242156,1016.25388395628,2811.45945015998,4258.16026060875,4102.36474609375
2005-01-01,44123.4070679055,37812.8910174761,42131.8270714745,36382.5079164537,34768.1759037525,56243.947298687,34520.2396492496,34460.5429970479,41994.7135305232,48799.7442261767,...,8021.54706395971,7561.8305480446,22560.1472939077,2702.23770075461,11223.3993032598,13430.6605521291,1133.10546057176,3448.53827356214,5086.62776073134,5323.455078125
2006-01-01,46302.0008800056,35991.5423923387,44537.7123351892,40504.0607253203,36470.2128399313,59008.993014024,36353.8803343631,36570.7630699009,44900.9381441374,52027.1890709689,...,9035.53178753546,9418.0627175215,24821.9367451879,3123.32015935927,11489.5608654088,15261.8088483267,1332.33941000282,3782.60350413031,5890.97800169795,6920.19970703125
2007-01-01,48050.2237771135,35779.0156794464,50438.229599278,44659.8951408034,41557.6235648179,64991.005147337,41640.0808695147,41023.7548562293,51799.2085521047,58487.0460144465,...,11254.3140875349,10461.3660887507,28863.9732885018,3572.36358879114,13940.9067455267,18466.5782621236,1586.47292057972,4762.08297292255,7210.59554755899,9101.2392578125
2008-01-01,48570.0459804586,39876.3062956324,47432.8790970463,46710.5055759013,45515.9617533473,74175.8776377701,45612.7106221441,49681.2190739771,57879.9437553916,64321.6744842874,...,13995.7830962135,10783.1708210632,32127.9831943287,4184.88598248749,15772.3270292979,22804.6222045964,1941.89997269337,5534.68944688199,8977.50685093365,11635.2841796875


## 3. Labour

In [53]:
df = read_world_bank_indicator(all_data, indicator='Population, total',
                               filename='Population')
save_to_database('Population', df)
df

Data coverage since 1999 (50 countries): 98.0 %
Data coverage since 2013 (30 countries): 96.7 %

Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland',
       'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria',
       'Portugal', 'Eurozone', 'China', 'India', 'Korea', 'Brazil',
       'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia',
       'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye',
       'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary',
       'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Taiwan'], dtype='object')

Missing first:

Missing last:

Done
Population: Inserted 1176 records


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,279040000,126631000,58682466,30401286,60504420,7143991,82100243,18812264,15812088,5321799,...,38660271,15176410,10761698,26252239,10237530,10283860,69907887,38585033,36653031,147214776
2000-01-01,282162411,126843000,58892514,30685730,60921384,7184250,82211508,19028802,15925513,5339616,...,38258629,15351799,10805808,26654439,10210971,10255063,71371371,39215135,37070774,146596869
2001-01-01,284968955,127149000,59119673,31020902,61367388,7229854,82349925,19274701,16046180,5358783,...,38248076,15523978,10862132,27014909,10187576,10216605,72854261,39837875,37480493,145976482
2002-01-01,287625193,127445000,59370479,31360079,61816234,7284753,82488495,19495210,16148929,5375931,...,38230364,15693790,10902022,27334503,10158608,10196916,74393759,40454050,37885028,145306497
2003-01-01,290107933,127718000,59647577,31644028,62256970,7339001,82534176,19720737,16225302,5390574,...,38204570,15859112,10928070,27623341,10129552,10193998,75963322,41057687,38278164,144648618
2004-01-01,292805298,127761000,59987905,31940655,62716306,7389625,82516260,19932722,16281779,5404523,...,38182222,16017966,10955141,27893911,10107146,10197101,77522427,41648268,38668796,144067316
2005-01-01,295516599,127773000,60401206,32243753,63188395,7437115,82469422,20176844,16319868,5419432,...,38165445,16175311,10987314,28147267,10087065,10211216,79075310,42220940,39070501,143518814
2006-01-01,298379912,127854000,60846820,32571174,63628261,7483934,82376451,20450966,16346101,5437272,...,38141267,16334575,11020362,28381078,10071370,10238905,80629670,42772910,39476851,143049637
2007-01-01,301231207,128001000,61322463,32889025,64021737,7551117,82266372,20827622,16381696,5461438,...,38120560,16495538,11048473,28600387,10055780,10298828,82218755,43306582,39876111,142805114
2008-01-01,304093966,128063000,61806995,33247118,64379696,7647675,82110097,21249199,16445593,5493621,...,38125759,16661462,11077841,28806185,10038188,10384603,83844783,43815313,40273769,142742366


## 6. Trade

In [54]:
df = read_world_bank_indicator(all_data, indicator='Current account balance (% of GDP)',
                               filename='Current account')
save_to_database('Current Account Pct of GDP', df)
df

Data coverage since 1999 (50 countries): 91.6 %
Data coverage since 2013 (30 countries): 96.7 %

Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland',
       'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria',
       'Portugal', 'China', 'India', 'Korea', 'Brazil', 'Saudi Arabia',
       'South Africa', 'Mexico', 'Thailand', 'Indonesia', 'Malaysia', 'Qatar',
       'Kuwait', 'Turkiye', 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru',
       'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Eurozone', 'Taiwan', 'United Arab Emirates'], dtype='object')

Missing first:
Belgium 2002-01-01 00:00:00
Ireland 2005-01-01 00:00:00
New Zealand 2000-01-01 00:00:00
Austria 2005-01-01 00:00:00
Qatar 2011-01-01 00:00:00

Missing last:

Done
Curren

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,-2.97598180084223,2.46175310702324,-2.11353382046635,0.124607703811068,3.39499471898115,9.49477188282003,-1.42985911872026,-5.59740342730537,3.50782246578161,1.71225041068949,...,-7.34397013599592,0.131405437058336,-5.1155828729177,-2.74952502384786,-8.37316170546344,-2.24901467913644,-1.80221287386216,0.777689613904397,-4.21229506178187,11.6664565462153
2000-01-01,-3.92088614748242,2.6304554483536,-1.84712992495041,2.48322832733606,1.18077857733456,10.4899126454199,-1.7404661031122,-3.72429027623484,1.73836824467618,1.37784217454517,...,-6.00567464124102,-1.14700034952076,-7.52537978913507,-2.9874113727773,-8.83884703380859,-4.35070087214669,-0.972570273448986,0.846461525662989,-3.15992237703676,17.4742424828314
2001-01-01,-3.72407498836611,1.97087169355986,-1.80557968729276,2.13242480189661,1.5218142190844,6.66593441613666,-0.364863756166508,-2.10538388216154,2.27392637964921,2.94195046736942,...,-3.11463011868753,-1.53826894255788,-6.89835566386782,-2.31175794610238,-6.1202055106571,-4.82618558892486,-0.401718427655422,-1.0593767406628,-1.40694788500181,10.4545347443772
2002-01-01,-4.17333961025019,2.60256560577514,-2.03639086013187,1.64503032397307,1.175629306184,6.8724559921272,1.91103265835049,-4.04533012402292,2.3281860989863,1.93714143070177,...,-2.78494396796903,-0.825199468328001,-6.20816077609579,-2.02693936398421,-6.45098818485992,-5.18860410860461,0.730979150171549,-1.31623163833537,8.97078467784247,7.9523289090993
2003-01-01,-4.55898086099347,3.0839379549121,-1.88403606478649,1.16498813828327,0.866601323987478,10.7914279174848,1.43846913459741,-6.26906085398024,5.15317712942085,3.19264763643285,...,-2.51252519913782,-0.290419263046469,-6.33137485844822,-1.61504289543387,-8.25455096874065,-5.78025583580088,4.66231377245509,-0.96359862609051,6.37990680177011,7.69799200492828
2004-01-01,-5.20485343641858,3.71938350458585,-2.36203193542683,2.26315076447856,0.432452993011335,12.4877045205757,4.53055764563467,-6.72068682864578,6.73082200775295,2.36333856212672,...,-5.99630095832025,2.84739040725261,-5.59942022897061,0.0288861085400799,-9.07203733400082,-3.72008813249091,4.97788419534309,-0.661483096041819,1.95058324319273,9.908322542702
2005-01-01,-5.74598170416952,3.52114102912066,-2.03429445027245,1.86965618692434,-0.00649497362518162,11.3634345831973,4.67487148183906,-6.23365908438409,6.06556937545051,4.16210292587251,...,-3.3141059735697,1.49174322427829,-7.34996387367016,1.50881026119421,-9.87694875083312,-2.04873491487123,2.34685756731662,-1.33773108955825,2.65363142179073,11.0454168411951
2006-01-01,-5.91101922107059,3.79586210941397,-3.16442995319484,1.36365624553912,0.0221640824996735,12.4843731667523,5.79311003542774,-6.0593853079141,7.79269553472333,3.34468921594548,...,-4.66620933546281,4.5529264424736,-10.8121877663106,3.24050716695869,-7.27842222201558,-2.5572614033307,2.45322162862231,-1.8486662318406,2.79449564264943,9.3254450282857
2007-01-01,-5.08872774231253,4.62330162774973,-3.78605086652334,0.752359497275174,-0.327690864015585,8.32016970609657,6.88284344098742,-7.50947574689242,5.89865319930189,1.51570591961567,...,-6.69478099515961,4.30154750471149,-13.9789444009951,1.42931463476859,-7.28307096663554,-4.70366256243025,0.315552631578947,-2.98192666963852,2.1036160391819,5.55457627319938
2008-01-01,-4.71586121002055,2.78294218549251,-3.98779625784346,0.204634001322009,-0.961095349295913,1.32642955584366,5.69385837100298,-4.9372314815571,4.09327586137291,2.9468952920735,...,-6.77192839796481,-3.72837458535564,-14.4129814805407,-4.41112077807089,-7.02666669121284,-1.86133326423192,-0.868821887213847,-2.67924104556952,1.49940141614234,6.25797161157479


In [55]:
df = read_world_bank_indicator(all_data, indicator='Current account balance (BoP, current US$)',
                               filename='Current account US Dollars')
save_to_database('Current Account', df)
df

Data coverage since 1999 (50 countries): 93.5 %
Data coverage since 2013 (30 countries): 96.7 %

Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland',
       'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria',
       'Portugal', 'Eurozone', 'China', 'India', 'Korea', 'Brazil',
       'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia',
       'Malaysia', 'Qatar', 'Kuwait', 'Turkiye', 'Philippines', 'Poland',
       'Chile', 'Greece', 'Peru', 'Hungary', 'Czechia', 'Egypt', 'Colombia',
       'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Taiwan', 'United Arab Emirates'], dtype='object')

Missing first:
Belgium 2002-01-01 00:00:00
Ireland 2005-01-01 00:00:00
New Zealand 2000-01-01 00:00:00
Austria 2005-01-01 00:00:00
Qatar 2011-01-01 00:00:00

Missing last:

Done

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,-286622000000,114126431432.297,-35706203345.0974,845351118.081352,50692422641.2379,28283270188.0401,-31384625220.8797,-21795709492.9592,15681703602.7107,3047192196.55612,...,-12487000000,99338040.6829709,-7294252083.21298,-1379913047.79369,-4108992898.71996,-1465751162.91064,-1634800000,670982414.631348,-11942825328.0158,22855420000
2000-01-01,-401928000000,130690474028.488,-30774085340.9977,18494424438.8816,16125180558.2182,29288780969.439,-33903966248.29,-15487273891.4343,7257328230.22331,2261835981.64661,...,-10343000000,-897526442.929414,-9817441650.67537,-1545828520.42069,-4173563161.65006,-2689755313.83905,-971000000,845404083.052557,-8980617892.62762,45382380000
2001-01-01,-394079000000,86219920754.3291,-29769769802.1206,15757824510.9175,20965385279.5954,19103243195.9331,-7099486034.16705,-7986938465.21441,9813967287.13199,4848072420.25253,...,-5946000000,-1100127918.63704,-9403099987.55552,-1202811329.86012,-3289609794.38738,-3272573493.81492,-388400000,-1040314751.93966,-3780423241.19359,32053820000
2002-01-01,-456109000000,108861295892.018,-36365488652.6254,12512912205.0114,17651008712.0079,21256501853.0357,39720517927.6966,-16002234756.2014,11032388748.3938,3460409556.63061,...,-5544000000,-580075884.957633,-9595594260.97012,-1110307794.82469,-4361443385.9147,-4264835726.29032,622400000,-1289193776.36478,8766610000,27472950000
2003-01-01,-522297000000,139380520054.084,-38749056160.2345,10432942306.8006,15984849589.3335,39074498538.4175,35985324846.9412,-29307731701.3676,29892053162.2424,6963066045.36346,...,-5473000000,-222193545.398375,-12812812179.9253,-948531329.398338,-7039900025.20667,-5785485093.44665,3743300000,-911997622.355946,8139930000,33128110000
2004-01-01,-635887000000,181993749574.64,-57233152147.3247,23235547976.4209,9166417134.83922,50439617363.6772,127505924403.463,-41286950910.194,44314391423.3045,5940838083.38819,...,-15297000000,2821176352.4679,-13492562448.2979,19286880.1631118,-9445878133.34234,-4457211225.0831,3921700000,-774546542.895837,3211790000,58559840000
2005-01-01,-749230000000,170122750083.948,-51768993007.7851,21933097496.4069,-142691013.40639,47532417798.3125,133087243134.907,-43342943309.5636,41570269413.2386,11007384393.5368,...,-10146000000,1824625140.66785,-18218753983.7318,1147610228.96893,-11181808084.8179,-2809704225.91577,2102800000,-1947743523.40683,5273750000,84388750000
2006-01-01,-816642000000,174672769353.091,-85755360799.5345,17990236971.1871,514325562.52478,55133415011.774,173486477217.385,-45318593059.6826,57194899544.2721,9461656812.10659,...,-16081000000,7004224398.15737,-29576385911.1083,2872489024.18697,-8422271299.47517,-3996084353.2344,2635400000,-2991011803.11064,6498802520.20453,92315570000
2007-01-01,-736554000000,211735646402.301,-117102419592.911,11050809835.8733,-8718514442.63438,40831622534.0376,235777197044.801,-64163007801.9241,50053545967.9083,4841519012.77284,...,-28722000000,7423002524.34518,-44579249236.0973,1460344785.97979,-10209896208.3164,-8945618935.57797,411600000,-6149613557.24906,6048537892.88669,72193020000
2008-01-01,-696526000000,142115935666.067,-116909573725.291,3177944944.24201,-28163013357.7814,7524476800.82068,213250033110.591,-52121667002.2045,38962664854.2205,10413116838.3802,...,-36135000000,-6698524204.55871,-51297053502.0193,-5317632556.54802,-11125011068.8044,-4407952631.4357,-1414600000,-6497270736.68045,5421206328.61008,103935400000


## 7. Government

In [56]:
df = read_world_bank_indicator(all_data, indicator='Expense (% of GDP)',
                               filename='Government Expense')
save_to_database('Government Expense', df)
df

Data coverage since 1999 (50 countries): 78.1 %
Data coverage since 2013 (30 countries): 81.7 %

Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Italy', 'Singapore', 'Finland', 'Belgium', 'Norway',
       'Israel', 'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Eurozone',
       'India', 'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico',
       'Thailand', 'Indonesia', 'Malaysia', 'United Arab Emirates', 'Kuwait',
       'Turkiye', 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru',
       'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Index(['Hong Kong', 'China', 'Taiwan', 'Qatar'], dtype='object')

Missing first:
Spain 2012-01-01 00:00:00
New Zealand 2001-01-01 00:00:00
Brazil 2010-01-01 00:00:00
Saudi Arabia 2010-01-01 00:00:00
Kuwait 2001-01-01 00:00:00
Turkiye 2008-01-01 00:00

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,18.1044378542979,17.3108900923912,33.7170904563521,19.7560910273366,45.3891466018177,19.2106707982503,31.6223512731369,24.5184972995938,37.7734176641817,38.1758420989158,...,38.7042829822247,18.7140038461769,41.7927585050364,17.8181315090752,43.9370207430329,34.8137558073008,None,22.1201943516985,16.7436856974566,20.2861006003545
2000-01-01,17.9265371411288,17.1095576406981,34.1554185060433,18.7745182688509,44.1329000361832,18.516531788552,31.3132203936295,24.0475111755467,36.6735470910849,36.7934107533369,...,38.3163011050351,20.1386805049182,42.8468439986057,18.4212887077482,41.3571337007691,33.5883876596674,None,16.9331428734368,16.8328883767368,21.1628915021522
2001-01-01,19.5097628135138,16.2333992095233,34.4873018353774,18.7110049059225,44.374324535171,17.5202183407864,30.7393649829232,26.1969228854032,37.204621057896,36.6254755765814,...,39.8463050254111,20.0571230559933,42.4340438802785,18.2439858212758,40.939494892426,36.6830856654541,None,None,16.9923529034125,22.7585313017298
2002-01-01,20.0840096450444,16.1070602104528,35.8182649819767,17.963084125068,45.3515498205411,19.4819504580923,31.1779156733936,26.3711684924223,37.3855851793023,36.6384893476224,...,39.7092391906566,20.0926250802076,41.9978460396771,17.6134329055327,42.9174917612905,37.0092033317205,27.2483504882555,None,19.6771706443151,22.5352476801625
2003-01-01,20.6606326076555,15.7293776764486,36.9706471907436,18.0373335713637,45.6356507095874,18.3415985235915,31.5756227476408,25.7176458276753,38.3615764123165,37.0814365888419,...,40.6280100855572,19.3366626993026,42.0223251231969,17.7708378805601,42.6711254896822,38.2850850660822,27.3324550898204,27.622281719396,19.8373029880995,23.046306391951
2004-01-01,20.4485355966129,15.0737560136373,37.4259116531411,17.2659172104417,45.0159798687691,18.1701679230064,30.3906705797076,25.619485421829,37.5836504271793,36.8121318883923,...,37.9937219413755,18.2083109080648,43.4528260324257,17.3833715752298,41.6103479388665,34.6353135726179,26.8903770863383,None,16.8763416365527,21.5738426684628
2005-01-01,20.7451029772761,15.0518407348066,37.8601543393565,17.8239879344923,45.1413507521639,17.8222313074719,30.4945571185722,25.5686400858015,36.8949486551591,34.8998439246717,...,37.0614317587554,17.2109749859784,42.9431876899722,18.0201336794962,41.787298993944,33.478623849944,27.3888579387187,None,None,19.9480998631588
2006-01-01,20.5951351231726,14.8453333163917,38.11643786787,17.0276912468077,44.7043320594475,16.8917221176105,29.1920187163533,24.9160079447074,37.8989848532023,33.2287477094202,...,36.7936781335121,15.8867269551126,42.4195973839682,17.0776767262438,42.8299602606799,32.6028263201167,32.7636716852841,None,None,19.500541407903
2007-01-01,20.8646452748144,14.0037943063894,38.4024500062104,16.9627803202211,44.1951595788519,16.3543043101587,28.0020803744674,24.463559144488,37.2703780867936,36.0667508169168,...,35.14342640218,16.0414047722644,44.0495839110452,18.1434002111455,42.375142656536,32.3566866768596,29.3352309344791,None,None,23.0065326912153
2008-01-01,22.7876261253222,15.2644121011595,41.2123754575206,17.0833534495245,44.6826820184904,15.8397851197895,28.2725241410726,24.332380954804,37.6791955475759,36.760980329934,...,35.9419277503926,18.4111324189756,47.2109656916541,17.8236305892688,44.2888579881118,32.0802600139505,30.3182043551089,24.5684126038182,None,21.5321667594705
